<a href="https://colab.research.google.com/github/markkinuthia/-blockchain-developer-bootcamp-final-project/blob/main/Yearn_TVL_%26_ROI_%5B0%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#@title
#Imports & settings
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
sns.set_theme(style="ticks", color_codes=True)
%matplotlib inline
%load_ext rpy2.ipython
%R options(tidyverse.quiet = TRUE)
%R options(lubridate.quiet = TRUE)
%R options(jsonlite.quiet = TRUE)
%R suppressMessages(library(tidyverse))
%R suppressMessages(library(lubridate))
%R suppressMessages(library(jsonlite))
%R suppressMessages(options(dplyr.summarise.inform = FALSE))

dplyr.summarise.inform,[RTYPES.NILSXP]


In [8]:
#@title
%%R
#Grab base query from Flipside
df = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/79667064-b29f-4297-9a9e-c8e162beb2fa/data/latest', simplifyDataFrame = TRUE)

#Change the date to date format
df$DATE <- parse_datetime(df$DATE)

#Rename & reorder columns
df = df %>% rename(asset_symbol = ASSET_SYMBOL, burn_amount = BURN_AMOUNT, date = DATE, 
                   mint_amount = MINT_AMOUNT, net_ytoken_increase = NET_YTOKEN_INCREASE,
                   ytoken_price = YTOKEN_PRICE, pricing_symbol = PRICING_SYMBOL, token_price = TOKEN_PRICE,
                   vault_name = VAULT_NAME, vault_symbol = VAULT_SYMBOL, 
                   ytoken_price = YTOKEN_PRICE, exposure = EXPOSURE) %>%
  select(date, vault_name, vault_symbol, asset_symbol, exposure, mint_amount, burn_amount,
         net_ytoken_increase, ytoken_price, pricing_symbol, token_price)
  
###Fill in the missing data
#create a date sequence from min date to max date
full_date_range <- tibble(date = seq(min(df$date), max(df$date), by = "days"))

#create a new empty df
df_full_dates <- df %>% filter(vault_symbol == "yeehaa")

#create the list of vaults to iterate over
vault_list <- df %>%  distinct(vault_symbol) 

#iterate over the vaults and fill the missing bits - I know I should vectorise this, please don't be disappointed in me
for(vault in vault_list$vault_symbol) {

  #filter to a single vault
  single_vault <- df %>% filter(vault_symbol == vault)
  
  #store the min_date
  min_date <- min(single_vault$date)
  
  #join on the full date range 
  single_vault <- full_date_range %>%
    left_join(single_vault, by = "date")
  
  #chop off data prior to min_date
  single_vault <- single_vault %>% filter(date >= min_date)
  
  #fill chars with above, prices with above, mint/burns with zero
  single_vault <- single_vault %>%
    fill(vault_name) %>%
    fill(vault_symbol) %>%
    fill(asset_symbol) %>%
    fill(exposure) %>%
    fill(pricing_symbol, .direction = "downup") %>%
    fill(ytoken_price, .direction = "downup") %>%
    fill(token_price, .direction = "downup") %>%
    replace_na(list(mint_amount = 0, burn_amount = 0, net_ytoken_increase = 0))
  
  #append the newly fixed data to the 
  df_full_dates <- df_full_dates %>%
    bind_rows(single_vault)

}

#update the master df
df <- df_full_dates  

### Calculate ROIs
#7 Day ROI - define ROI as the annualised 7 day vault return - assume 52 weeks/year
#calculate the 7 day lagged value of ytoken_price
tmp <- df %>%
  select(vault_name, date, ytoken_price) %>%
  group_by(vault_name) %>%
  mutate(ytoken_lag7 = lag(ytoken_price, n=7, order_by = date)) %>%
  ungroup()
df <- df %>% bind_cols(tmp %>% select(ytoken_lag7))

#Calculate 7 day ROI
df <- df %>%
  mutate(vault_7day_ROI = (ytoken_price - ytoken_lag7) * 52 * 100)

# ROI from inception
#Generate min ytoken price & inception date by vault
tmp <- df %>%
  select(vault_name, date, ytoken_price) %>%
  group_by(vault_name) %>%
  mutate(ytoken_inception_price = min(ytoken_price),  #price is always increasing according to docs
         inception_date = min(date),
         days_since_inception = as.numeric(as.duration(inception_date %--% date), "days")
         ) %>%  
  ungroup()
df <- df %>% bind_cols(tmp %>% select(ytoken_inception_price, inception_date))

#Calculate ROI since inception
df <- df %>%
  mutate(days_since_inception = as.numeric(as.duration(inception_date %--% date), "days"),
         vault_inception_ROI = (ytoken_price - ytoken_inception_price) / days_since_inception * 365 * 100
  )

### Calculate deposited tokens and TVL
#Generate a running total (cumsum) of mints & burns to get tokens issued
df <- df %>% 
  group_by(vault_name) %>%
  mutate(ytokens_on_issue = cumsum(net_ytoken_increase)) %>%
  ungroup()
  
#Calculate ytokens on issue and TVL
df <- df %>%
  mutate(asset_tokens_deposited = ytokens_on_issue * ytoken_price,
         total_assets_usd = asset_tokens_deposited * token_price)
  
#filter vaults with data errors (waiting fix from Flipside)
#df <- df %>% filter(!(vault_symbol %in% c("yvUSDT", "yvWBTC")))


#Tag top10 by TVL
top10_by_tvl <- df %>%
  filter(date == max(date)) %>%
  arrange(desc(total_assets_usd)) %>%
  head(10) %>%
  select(vault_name) %>%
  mutate(top10_tvl = 1)
df <- df %>%
  left_join(top10_by_tvl, by = "vault_name")
df$top10_tvl <- replace_na(df$top10_tvl, 0)

#Tag the top10 by 7 day ROI
top10_by_7roi <- df %>%
  filter(date == max(date)) %>%
  arrange(desc(vault_7day_ROI)) %>%
  head(10) %>%
  select(vault_name) %>%
  mutate(top10_7day_roi = 1)
df <- df %>%
  left_join(top10_by_7roi, by = "vault_name")
df$top10_7day_roi <- replace_na(df$top10_7day_roi, 0)

#Tag the top10 by inception ROI
top10_by_inceproi <- df %>%
  filter(date == max(date)) %>%
  arrange(desc(vault_inception_ROI)) %>%
  head(10) %>%
  select(vault_name) %>%
  mutate(top10_inception_roi = 1)
df <- df %>%
  left_join(top10_by_inceproi, by = "vault_name")
df$top10_inception_roi <- replace_na(df$top10_inception_roi, 0)


# Exploring Yearn - TVL and ROI

## Introduction to Yearn

Yearn is a DeFi yield aggregator protocol.  There are a million different ways to earn yield on your tokens in DeFi.  Some are risky, some are safe, and some are really complex.  Yearn aims to simplify this for users by providing a simple interface to deposit tokens and earn yield.  Yearn provides a series of vaults, based around a deposit asset type, in which users can deposit an asset to earn yield.  Behind the scenes, Yearn automates the allocation of funds from vaults into strategies, which use cutting edge yield-farming techniques to bring gains back to the vault.  Users can withdraw a larger number of tokens at a later date, with the gains representing the yield earned by the vault strategists.  There are a couple of key things to know about Yearn:
- Yields are earned in the denomination of the vault
- Users retain exposure to the deposited asset whilst it is in the vault

For example, you deposit ETH into a vault & withdraw a week later.  ETH price has dropped.  You will have more ETH than when you deposited, but the USD value of that ETH may be less than you had at the beginning.  This is actually an advantage for some users - if a user wants to maintain exposure to an asset (say ETH, or BTC, because they think it will go up in the longer term), they can deposit in a corresponding vault to earn yield whilst they are hodling.  Yearn also has a handy feature to further support this - you can deposit into any vault using ETH, WBTC, DAI, USDC, or USDT, and Yearn will automatically swap into the vault deposit asset.   

---

## Yearn Vaults v2

Yearn launched version 2 of its Vaults in July 2020.  These were a major upgrade on the original vaults, allowing up to 20 strategies to service a single vault.  Prior to v2, Yearn Vaults only had one strategy.  Most of the v2 Vaults have 2 or more strategies, with automatic logic working to allocate user funds to these strategies to get the best return fo the Vault.  This dashboard will focus on the top 20 Yearn v2 Vaults by TVL.  These vaults account for around US\$2.6b of value out of the US\$3.5b listed on [Defi Pulse](https://defipulse.com/yearn.finance).  There are over 100 different Yearn vaults & other products (iearn, v1 vaults), but these top v2 vaults represent most of the value locked in Yearn at the moment.  We will use the abbreviation TVL (Total Value Locked) to refer to the USD value deposited in Vaults.




### Growth in Value of V2 Vaults

The graph below shows the growth of the top 20 V2 vaults over 2021.  This growth has been a mix of organic protocol growth, and the migration of funds from obsolete vaults (v1 or older v2 vaults).  Note the falls in value which occurred in May & June 2021, when there were rapid drops in the value of most crypto assets agains the USD.

In [6]:
#@title
#Plot TVLs
df_p = %R df %>% filter(vault_symbol != "yvCurve-triCrypto") %>% group_by(date) %>% summarise(top10_tvl_by_date = sum(total_assets_usd))
fig = px.line(df_p, x="date", y="top10_tvl_by_date", 
              template="simple_white", width=800, height=800/1.618,
              title= 'Yearn v2 Vaults - Total Value of Top 20 Vaults in USD')
fig.update_yaxes(title_text='USD Value')
fig.show()

UsageError: Line magic function `%R` not found.


---
## Which Vaults Hold the Value?

The chart below shows the value by Vault for the top 10 vaults by value.  Before we take a good look, it is a good time to talk about vault names & what they mean.  You'll notice the vault names below all start with *yv* - it doesn't take a genius to figure that means *Yearn Vault*.  You'll then notice there are two types of names:
- *yv\<Token Name\>* - these are vaults where the deposited token (example - WETH for the yvWETH vault) are used for yield farming by the vault strategies
- *yvCurve-\<Curve Pool Name\>* - these vaults start by depositing the tokens into the corresponding Curve Pool, then yield farming with the resulting deposit token.  Curve is a dex, specialising in stablecoin swaps and big brained mathemetics.  Yearn uses Curve pools as the basis for many of their Vaults.

In the graph below, you'll see that the biggest pool by value is the USDC pool, followed closely by the DAI pool.  These are both USD pegged stablecoins.  In fact, so is IronBank, LUSD, USDT and USDN - 6 out of the top 10 vault assets.  There are three ETH based vaults, and the Yearn governance token, YFI, is also in the list.

An interesting feature of this chart is the large increases of the yvUSDC vault on the 7th of June, and the yvDAI vault on 29th of June.  The USDC transaction was a US\$180m deposit from what looks like a whale account, and the DAI transaction was a series Yearn protocol operations - possibly a migration of funds from an older version of the yvDAI vault to this one.

*Usage tip - The graphs in this dashboard are interactive.  Click on a plot trace in the legend once to turn it on & off.  Double-click on a plot trace in the legend to turn off everything but the selected plot.  Click & drag in the plot area to zoom.  Double-click the chart area to unzoom.*

In [5]:
#@title
#Plot ROI since inception
df_p = %R df %>% filter(top10_tvl == 1)
fig = px.line(df_p, x="date", y="total_assets_usd", color='vault_symbol',
              template="simple_white", width=800, height=800/1.618,
              title= 'Yearn v2 Vaults TVL (Top 10 Vaults Only)'
              )
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.update_yaxes(title_text='USD Value')
fig.show()

UsageError: Line magic function `%R` not found.


---
## Vault Return on Investment


### Defining ROI

Vault returns are defined as a Return on Investment - ROI.  This is usually an annualised number, which represents the returns you might get if you deposited in a vault for a year.  There are a number of caveats around this however - the returns often change over time, and the returns are denominated in the vault deposit asset.  This is a bit different to a traditional APR or APY definition, although [yearn.finance](http://yearn.finance) does use the APY label on it's ROI calculation on the main Vault deposit page.  It describes this as "annualised weekly yield" which is more or less the definition we will use.

Each Vault has an exchange rate built into it - deposit 1 token, get a bit less than one yvtoken in return.  Over time, 1 deposit token will buy less and less yvtokens - the price is always rising.  For a given investment, the ROI is determined by the following formula, where the Buy & Sell prices are the number of native tokens required to buy the yvtokens:

> $ROI_{annualised} = \frac{(SellPrice) - (BuyPrice)}{TimeInYears}$

For comparison of Vault returns we will use a rolling 7 day period to determine the ROI.  This matches the [yearn.finance](http://yearn.finance) approach and, let's face it, we are in DeFi and we have **very** short investment horizons.  More info on calculating ROI can be found at [Yearn Docs](https://docs.yearn.finance/resources/guides/how-to-understand-yvault-roi) and this [helpful notebook on Github](https://github.com/Zer0dot/yearn_roi/blob/master/yearn_vaults_ROI_calc.ipynb).

The graphs below are an example of the ROI calculations for the yvUSDC v2 Vault.  The top chart shows the price of a yvUSDC token in USDC, and the chart below it is the annualised 7 day ROI.  You will notice that the Vault yield travelled happily along between 15-25% until the May 2021 Crypto Crash, after which yields fell to around 5%.  This will be a recurring theme when we look at other Vault returns.

In [1]:
#@title
#Plot subplots of ytoken price & ROI
df_p = %R df %>% filter(vault_symbol == "yvUSDC")
fig = make_subplots(rows=2, cols=1, subplot_titles=("yvUSDC Price (in USDC)", "yvUSDC 7 Day Annualised ROI"))
fig.append_trace(go.Scatter(x=df_p["date"], y=df_p["ytoken_price"], name="Price"), row=1, col=1)
fig.append_trace(go.Scatter(x=df_p["date"], y=df_p["vault_7day_ROI"], name="ROI"), row=2, col=1)
fig.update_layout(width=800, height=800/1.618, title_text="yvUSDC Price & Returns")
fig.update_layout(template="simple_white", showlegend=False)
fig.update_yaxes(title_text='yvUSDC Price in USDC', row=1, col=1)
fig.update_yaxes(title_text='ROI % Annualised', row=2, col=1)
fig.show()

UsageError: Line magic function `%R` not found.


### ROI of the Largest Vaults
 
We have seen from above that a huge chunk of capital is tied up in the top 10 Yearn v2 Vaults.  Do these large vaults generate large returns for the depositors?  It turns out that it depends on the vault, and on the timing.

The two charts below show the ROI of the top 10 Yearn v2 Vaults by value.  The first chart shows the individual Vaults, and the 2nd is an average ROI of these vaults weighted by the vault value.

We see that most of these vaults have seen a steady reduction in ROI from earlier in the year through to the present.  Returns north of 20% were typical, but they have steadily declined to hover in the 0-10% band.  The average chart shows a slight recovery in yields over the last month, giving some hope for better returns in the future.  Of the individual Vaults, the yvCurve-stETH vault, the yvCurve-USDN vault and the yvCurve-IronBank vault appear to have been slightly higher returns than others over time.

*Tip - Use the zoom feature to better explore the difference between the vaults, particularly in later months.*

In [2]:
#@title
#Plot ROIs
df_p = %R df %>% filter(top10_tvl == 1)
fig = px.line(df_p, x="date", y="vault_7day_ROI", color='vault_symbol',
              template="simple_white", width=800, height=800/1.618,
              title= 'Yearn v2 Vaults ROI - Top 10 Vaults by TVL')
fig.update_yaxes(title_text='ROI % Annualised')
fig.update_yaxes(range=[0, 70])
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=1.1
))
fig.show()
df_p = %R df %>% filter(top10_tvl == 1) %>% group_by(date) %>% summarise(weighted_av = weighted.mean(vault_7day_ROI, total_assets_usd, na.rm = TRUE))
fig2 = px.line(df_p, x="date", y="weighted_av",
              template="simple_white", width=800, height=800/1.618,
              title= 'Yearn v2 Vaults ROI - Average of Top 10 Vaults by TVL')
fig2.update_yaxes(title_text='ROI % Annualised')
fig2.show()

UsageError: Line magic function `%R` not found.


### Which Vaults Have the Largest ROI?

Enough about the biggest vaults.  Let's see who's got the biggest **returns.**

We have the same set of charts below as the section above, but this time we are looking at the top 10 Vaults by ROI, instead of value.  We can see that, on average, the returns from these vaults are higher than the previous chart - currently 7.5% compared with 5.7%.  In earlier times, returns were **much** higher in the best performing vaults.

One vault stands out as being consistently higher than the the others, even when returns have dropped off.  The yvBOOST Vault, also known as the Yearn Compounding veCRV yVault, is a single strategy vault which accepts yveCRV-DAO tokens.  These tokens entitle the owner to a share of Curve's trading fees, which are harvested, swapped for more yveCRV-DAO and deposited back into the vault.  Thus the returns are auto compounding.  The results are impressive, with the ROI staying over 16% right through the current bearish market.

*Tip - Use the zoom feature to better explore the difference between the vaults, particularly in later months.*

In [4]:
#@title
#Plot ROIs
df_p = %R df %>% filter(top10_7day_roi == 1)
fig = px.line(df_p, x="date", y="vault_7day_ROI", color='vault_symbol',
              template="simple_white", width=800, height=800/1.618,
              title= 'Yearn v2 Vaults ROI - Top 10 Vaults by ROI')
fig.update_yaxes(title_text='ROI % Annualised')
fig.update_yaxes(range=[0, 70])
fig.update_layout(legend=dict(
    yanchor="top",
    y=1.1,
    xanchor="left",
    x=0.01,
    font=dict(
            size=11,
        )
))
fig.show()
df_p = %R df %>% filter(top10_7day_roi == 1) %>% group_by(date) %>% summarise(weighted_av = weighted.mean(vault_7day_ROI, total_assets_usd, na.rm = TRUE))
fig2 = px.line(df_p, x="date", y="weighted_av",
              template="simple_white", width=800, height=800/1.618,
              title= 'Yearn v2 Vaults ROI - Average of Top 10 Vaults by ROI')
fig2.update_yaxes(title_text='ROI % Annualised')
fig2.show()



UsageError: Line magic function `%R` not found.


### Returns since Inception

Just say you're a bored ape and you jump on every new Yearn vault on the day it launches.  What do the returns on the Yearn v2 Vaults look like since inception?  The graph below tells the story, showing the top 10 vaults by ROI since inception.  The clear winner is yvCurve-sETH, at 27% ROI since inception.  This vault relies on Curve & Convex protocols to earn staking rewards, which are swapped & reinvested back into the vault.  Another interesting datapoint is yvBOOST - it's the only vault which has seen the returns since inception steadily rising.

In [ ]:
#@title
#Plot ROIs
df_p = %R df %>% filter(top10_inception_roi == 1)
fig = px.line(df_p, x="date", y="vault_inception_ROI", color='vault_symbol',
              template="simple_white", width=800, height=800/1.618,
              title= 'Yearn v2 Vaults ROI Since Inception')
fig.update_yaxes(range=[0, 100])
fig.update_yaxes(title_text='ROI % Annualised')
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99
))
fig.show()

---
### Returns Based on Asset Exposure

As mentioned in the introduction, users *maintain exposure to the deposited asset* when they ape into a Yearn vault.  Yearn gives the users plenty of choice from this perspective.  There are vaults which take deposits of:
- ETH in the form of WETH, sETH, stETH and others
- BTC in the form of WBTC, sBTC, oBTC, pBTC and others
- Stablecoins - DAI, USDC, USDT, FRAX, EURS, alUSD and others
- DeFi tokens - YFI, eCRV-DAO, LINK and others

Is there a difference in the returns from Yearn Vaults between these asset classes?  The graphs below attempts to answer this question.  The first chart below shows the volume weighted average ROI of the vaults in each of the asset exposure groups.  

Stablecoin yields have consistently been stronger over the period of this chart.  Whilst they have decreased substantially from the 15-25% in the midst of the crypto price boom, they have settled at a respectable 7% in recent weeks.  ETH denominated vaults have followed the same trend, albeit at lowe roverall levels.  ETH returns have settled down at around 2% - better than you get from a savings account, but low in the realms of DeFi.  Our DeFi tokens have performed somewhere in between.  With only three tokens in the pool, this is more to do with these vaults than generally about DeFi. BTC based vaults are low, but have been steadily increasing from 1.5% through to the current range of 4-5%.  Interestingly, BTC returns have outpaced ETH returns from the end of June through to the present.  This is a rare case of BTC flippening ETH - don't tell the BTC Maxies!

The 2nd chart below shows all of the individual vaults in the dataset, split by underlying asset exposure.   Here are some observations:
- The ETH value has been driven by the yvCurve-SETH pool - it has returned significantly more than the other ETH denominated pools, even in recent weeks when crypto pricing has been depressed.  The underlying Curve pool earns boosted CRV rewards which probably accounts for much of this good performance.
- yvBOOST accounts for most of the ROI in the DeFi tokens, being the best performing vault in the dataset.  The yYFI vault in particular has a very low return.  Users probably aren't buying YFI for the yield in Yearn, and are happy to get some return whilst they hodl for capital appreciation.
- The stablecoins aren't easy to differentiate - current yields range from 5-12% and the best yielding vault changes often.  
- BTC returns are mainly driven by two pools - pBTC and oBTC.  The native BTC pool - yvWBTC - provides very modest returns, ranging between 0.5% and 2%. 



In [ ]:
#@title
#Plot TVLs
df_p = %R df %>% filter (date >= ymd('2021-05-01')) %>% filter(vault_symbol != "yvCurve-triCrypto") %>% group_by(date,exposure) %>% summarise(weighted_av = weighted.mean(vault_7day_ROI, total_assets_usd, na.rm = TRUE)) 
fig = px.line(df_p, x="date", y="weighted_av", color='exposure',
              template="simple_white", width=800, height=800/1.618,
              title= 'Yearn v2 Vaults ROI by Underlying Asset Exposure')
fig.update_yaxes(title_text='ROI % Annualised')
#fig.update_yaxes(range=[0, 70])
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99
))
fig.show()

In [ ]:
#@title
df_p = %R df %>% filter(vault_symbol != "yvCurve-triCrypto") %>% filter (date >= ymd('2021-05-01'))
fig = px.line(df_p, x="date", y="vault_7day_ROI", color='vault_symbol', facet_row='exposure',
              template="simple_white", width=1000, height=1200/1.618,
              title= 'Yearn v2 Vaults ROI by Underlying Asset Exposure - Individual Vaults')
fig.update_yaxes(title_text='ROI % Annualised')
fig.show()

---

## Conclusions

Our exploration of Yearn has uncovered a number facts.  Most of the Yearn TVL is accounted for by v2 Vaults, and the biggest v2 Vaults contain the popular stablecoins DAI and USDC.  Yearn has a close relationship with Curve, and uses Curve pools in many of the Vaults & Strategies to earn yield for users.  Yearn ROI is calculated from the price of the yvToken at different points in time.  We have seen that this ROI is volatile - past peformance is not a prediction of future returns!  We have seen that the largest vaults by value are not always the vaults with the best ROI.  We also saw that the yvBOOST vault is a standout in terms of its investment returns, consistently outperforming other vaults and even improving returns over time.  Finally, we took a look at which asset class has the best returns over time - stablecoins consistently provide better returns than most other asset classes.

*Data in this notebook was sourced from [Flipside Crypto](https://flipsidecrypto.com).*

# Interactive Feature - TVL and ROI for Any Vault

In [12]:
#@title # Pick-a-Vault TVL & ROI { run: "auto" }
#@markdown Select a Vault to see the TVL & ROI chart for that Vault:
selected_vault = "yvCurve-oBTC" #@param ["yvBOOST", "yvCurve-alUSD", "yvCurve-EURS", "yvCurve-FRAX", "yvCurve-IronBank", "yvCurve-LINK", "yvCurve-LUSD", "yvCurve-oBTC", "yvCurve-pBTC", "yvCurve-sBTC", "yvCurve-sETH", "yvCurve-stETH", "yvCurve-triCrypto", "yvCurve-USDN", "yvDAI", "yvUSDC", "yvUSDT", "yvWBTC", "yvWETH", "yvYFI"]
#@markdown *If the charts don't automatically update, re-run the notebook from the Runtime->Run All option in the menu bar.  You may also need to click the play button at the top left.  You should only have to do this once per session.*
#Plot subplots of ytoken price & ROI
df_p = %R df 
df_p = df_p[df_p.vault_symbol==selected_vault]
fig = make_subplots(rows=2, cols=1, subplot_titles=(selected_vault + " USD Value", selected_vault + " 7 Day Annualised ROI"))
fig.append_trace(go.Scatter(x=df_p["date"], y=df_p["total_assets_usd"], name="TVL in USD"), row=1, col=1)
fig.append_trace(go.Scatter(x=df_p["date"], y=df_p["vault_7day_ROI"], name="ROI"), row=2, col=1)
fig.update_layout(width=800, height=600, title_text=selected_vault + " TVL & ROI")
fig.update_layout(template="simple_white", showlegend=False)
fig.update_yaxes(title_text='Value in USD', row=1, col=1)
fig.update_yaxes(title_text='ROI % Annualised', row=2, col=1)
fig.show()
